
---

# 📊 Proyecto Final: Inteligencia de Negocios y Minería de Datos

## 📘 Informe de Proyecto Final

**Materia:** Inteligencia de Negocios (ICC-321-T)
**Tema:** Dashboard Interactivo y Modelo de Minería de Datos Descriptivo

**Autores:**

* Randy Alexander Germosén Ureña *(1013-4707)*
* Fernando Almonte Delgado *(1015-7628)*

**Repositorio:**
[icc321-2025-final](https://github.com/TZeik/icc321-2025-final) <img src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" width="15" height="15"/>

---

## 🎯 Objetivo del Proyecto

El propósito de este proyecto es desarrollar una solución integral de Inteligencia de Negocios utilizando datos públicos del gobierno de la República Dominicana. Consta de dos componentes principales:

1. **Dashboard Interactivo:** Permite visualizar y monitorear métricas de gasto y nómina para apoyar la toma de decisiones.
2. **Modelo de Minería de Datos:** Implementación de un modelo descriptivo (Clustering) para descubrir patrones y segmentar perfiles de empleados.

---

## 📂 Datasets Utilizados

Se procesaron y unificaron datos históricos abarcando el periodo **2018–2025**:

1. **Nómina de la Contraloría General de la República:**
   Información detallada sobre empleados, cargos, departamentos y sueldos.
2. **Índice de Precios al Consumidor (IPC):**
   Datos del Banco Central utilizados para calcular el **salario real** (ajustado por inflación) en comparación con el salario nominal.

---

## 🧠 Metodología

El desarrollo del proyecto se estructuró en las siguientes fases técnicas:

### 1. Ingeniería de Datos (ETL)

* **Extracción y Limpieza:**

  * Unificación de múltiples archivos CSV mensuales/anuales.
  * Estandarización de nombres de cargos, normalización de formatos monetarios y corrección de codificación (`latin-1`, `utf-8`).
  * Homogeneización de los nombres de los meses.
* **Enriquecimiento:**

  * Cruce entre nómina e IPC para calcular la pérdida de poder adquisitivo.

### 2. Almacenamiento (Data Warehousing)

* Implementación de un **Data Warehouse** local con **SQLite**.
* Diseño bajo un **Esquema en Estrella**, con:

  * Tabla de hechos: `fact_nomina`
  * Tablas de dimensiones: `dim_empleado`, `dim_tiempo`

### 3. Visualización (Dashboard)

* Creación de Dashboard interactivo subido a **Tableau Public**.

### 4. Minería de Datos (Machine Learning)

* **Preprocesamiento:**
  Codificación de variables categóricas y escalado numérico.
* **Modelado:**
  Aplicación de **K-Means Clustering** para identificar grupos de empleados con características similares.
* **Evaluación:**

  * Método del Codo
  * Coeficiente de Silueta

---

## 📊 Resultados Principales

La solución permite analizar hallazgos relevantes como:

* Diferencias entre **Sueldo Nominal** y **Sueldo Real** a lo largo del tiempo.
* Identificación de departamentos con mayor incremento en el gasto de nómina.
* Clusters de empleados basados en sueldo, cargo y antigüedad, revelando patrones ocultos en la organización.

---

## 🧩 Herramientas Utilizadas

### Lenguajes y Entorno

* **Python 3.x** (Jupyter Notebook)

### Librerías Principales

* `pandas` — Manipulación y limpieza de datos
* `sqlite3` — Data Warehouse local
* `scikit-learn` — Algoritmo K-Means y métricas
* `matplotlib` — Visualización del método del codo

### Visualización

* **Tableau Public** — Dashboard interactivo final

---



In [1]:
import pandas as pd
import sqlite3
import numpy as np
import calendar
from pandas.tseries.offsets import MonthEnd

## 1. Configuración del Entorno y Reglas de Negocio
Importación de librerías esenciales (`pandas`, `sqlite3`, `numpy`) y definición de funciones auxiliares para la limpieza de datos monetarios, estandarización de texto y el cálculo estimado del Impuesto Sobre la Renta (ISR) basado en la escala salarial local.

In [2]:
meses_map = {
    'ENERO': 1, 'FEBRERO': 2, 'MARZO': 3, 'ABRIL': 4, 'MAYO': 5, 'JUNIO': 6,
    'JULIO': 7, 'AGOSTO': 8, 'SEPTIEMBRE': 9, 'OCTUBRE': 10, 'NOVIEMBRE': 11, 'DICIEMBRE': 12
}
meses_inv_map = {v: k for k, v in meses_map.items()}

# --- FUNCIONES DE LIMPIEZA ---
def limpiar_moneda(valor):
    if pd.isna(valor): return 0.0
    s = str(valor).replace('RD$', '').replace(',', '').strip()
    try: return float(s)
    except: return 0.0

def limpiar_texto(texto):
    if pd.isna(texto): return "DESCONOCIDO"
    return str(texto).strip().upper()

def clean_float(val):
    try: return float(str(val).strip())
    except: return 0.0

def normalizar_estatus(texto):
    """Normaliza las categorías de Estatus según reglas de negocio"""
    t = str(texto).strip().upper()
    
    if t in ['FIJO', 'FIJOS']:
        return 'FIJO'
    elif t in ['CONTRATADO']:
        return 'CONTRATADO'
    elif t in ['PERSONAL DE CARACTER TEMPORAL', 'TEMPORAL', 'TEMPORAL EN CARGO DE CARRERA']:
        return 'TEMPORAL'
    elif t in ['PERSONAL DE SEGURIDAD', 'SEGURIDAD']:
        return 'SEGURIDAD'
    elif t in ['SUELDO AL PERSONAL NOMINAL EN PERIODO PROBATORIO', 'PRUEBA']:
        return 'PERIODO PROBATORIO'
    elif t in ['TRAMITE DE PENSIONES', 'TRAMITE DE PENSION']:
        return 'TRAMITE DE PENSION'
    else:
        return t

def calcular_isr(sueldo):
    if sueldo <= 34685: return 0.0
    elif sueldo <= 52027: return (sueldo - 34685) * 0.15
    elif sueldo <= 72260: return 2601 + ((sueldo - 52027) * 0.20)
    else: return 6648 + ((sueldo - 72260) * 0.25)

print("Entorno configurado exitosamente.")

Entorno configurado exitosamente.


## 2. Proceso ETL: Extracción, Limpieza y Normalización
Carga de los datasets crudos (IPC histórico y Nómina). Se realiza una limpieza profunda que incluye:
* Conversión de tipos de datos (fechas, monedas).
* **Normalización de Estatus Laboral:** Se unifican categorías dispersas (ej: "FIJOS" y "FIJO" -> "FIJO"; "PRUEBA" y "PROBATORIO" -> "PERIODO PROBATORIO", etc.) para garantizar consistencia.
* Enriquecimiento del dataset de IPC con métricas de variación mensual y acumulada.

In [3]:


# CARGA DE IPC
df_ipc = pd.read_csv('./raw_data/ipc_base_1984-2025.csv', sep=';')
df_ipc = df_ipc[df_ipc['PERIODO'] >= 2018].copy()

# Limpieza IPC
meses_map = {'ENERO': 1, 'FEBRERO': 2, 'MARZO': 3, 'ABRIL': 4, 'MAYO': 5, 'JUNIO': 6,
             'JULIO': 7, 'AGOSTO': 8, 'SEPTIEMBRE': 9, 'OCTUBRE': 10, 'NOVIEMBRE': 11, 'DICIEMBRE': 12}
df_ipc['MES_NUM'] = df_ipc['MES'].apply(limpiar_texto).map(meses_map)

cols_ipc = ['INDICE', 'VAR_MES', 'VAR_DIC', 'VAR_12', 'PROM_12']
for col in cols_ipc:
    df_ipc[col] = df_ipc[col].apply(clean_float)

df_ipc_clean = df_ipc[['PERIODO', 'MES_NUM'] + cols_ipc].rename(columns={
    'PERIODO': 'ANIO', 'INDICE': 'IPC', 'VAR_MES': 'inflacion_mensual',
    'VAR_DIC': 'inflacion_acumulada', 'VAR_12': 'inflacion_interanual'
})

# CARGA DE NÓMINA
df_nomina = pd.read_csv('./raw_data/nomina-empleados-fijos-y-contratados-CSV-2018-2025.csv', 
                        sep=';', encoding='latin-1', low_memory=False)

# Normalización Cabeceras
df_nomina.columns = (df_nomina.columns.str.strip().str.upper()
                     .str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf-8"))
df_nomina.columns = [c.replace('AO', 'ANIO') for c in df_nomina.columns]

# Transformaciones Básicas
df_nomina['SUELDO_NOMINAL'] = df_nomina['SUELDO'].apply(limpiar_moneda)
df_nomina['ANIO'] = pd.to_numeric(df_nomina['ANIO'].astype(str).str.extract(r'(\d{4})')[0], errors='coerce')
df_nomina['MES_NUM'] = df_nomina['MES'].astype(str).apply(limpiar_texto).map(meses_map)

# Limpieza Textos Generales
for col in ['NOMBRE', 'FUNCION', 'DEPARTAMENTO', 'ESTATUS']:
    df_nomina[col] = df_nomina[col].apply(limpiar_texto)
    df_nomina[col] = df_nomina[col].str.replace(r"[,.\"\'-]", " ", regex=True).str.replace(r"\s+", " ", regex=True).str.strip()
    df_nomina[col] = df_nomina[col].str.replace(r'\s+\bY\b\s+', ' ', regex=True)
    df_nomina['NOMBRE'] = df_nomina['NOMBRE'].str.replace('Ï¿½', 'Ñ', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('INGENIERÏ¿½A', 'INGENIERIA', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('ANÏ¿½LISIS', 'ANALISIS', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('ADMINISTRACIÏ¿½N', 'ADMINISTRACION', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('DISEÏ¿½ADOR', 'DISEÑADOR', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('GRÏ¿½FICO', 'GRAFICO', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('TRÏ¿½MITE', 'TRAMITE', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('PENSIÏ¿½N', 'PENSION', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('PERÏ¿½ODO', 'PERIODO', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('AUDITORÏ¿½A', 'AUDITORIA', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('AUDITORÏ¿½AS', 'AUDITORIAS', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('DIRECCIÏ¿½N', 'DIRECCION', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('DIVISIÏ¿½N', 'DIVISION', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('CAPACITACIÏ¿½N', 'CAPACITACION', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('MONSEÏ¿½OR', 'MONSEÑOR', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('UREÏ¿½A', 'UREÑA', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('NIÏ¿½EZ', 'NIÑEZ', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('PQUEÏ¿½', 'PQUEÑ', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('TRAUMATOLÏ¿½GICO', 'TRAUMATOLOGICO', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('QUIRÏ¿½RGICO', 'QUIRURGICO', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('PIÏ¿½A', 'PIÑA', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('CAFÏ¿½', 'CAFE', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('COMUNICACIÏ¿½N', 'COMUNICACION', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('CENTRALÏ¿½', 'CENTRAL', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('DESEMPEÏ¿½O', 'DESEMPEÑO', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('TÏ¿½CNICO', 'TECNICO', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('SECCIÏ¿½N', 'SECCION', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('TRANSPORTACIÏ¿½N', 'TRANSPORTACION', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('MAYORDOMÏ¿½A', 'MAYORDOMIA', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('ALMACÏ¿½N', 'ALMACEN', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('INFORMACIÏ¿½N', 'TRANSPORTACION', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('REVISIÏ¿½N', 'REVISION', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('CERTIFICACIÏ¿½N', 'CERTIFICACION', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('JURÏ¿½DICA', 'JURIDICA', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('TECNOLOGÏ¿½A', 'TECNOLOGIA', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('PLANIFICACIÏ¿½N', 'PLANIFICACION', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('EVALUACIÏ¿½N', 'EVALUACION', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('INVESTIGACIÏ¿½N', 'INVESTIGACION', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('INVESTIGACIÏ¿½', 'INVESTIGACION', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('ESTADÏ¿½STICOS', 'ESTADISTICOS', regex=False)
    df_nomina[col] = df_nomina[col].str.replace('CIIVLES', 'CIVILES', regex=False)
    
df_nomina['ESTATUS'] = df_nomina['ESTATUS'].apply(normalizar_estatus)

# INVERTIR RANGO > 72737 a 144741
r1 = df_nomina.loc[72737:144741, ['FUNCION', 'DEPARTAMENTO']]
df_nomina.loc[72737:144741, ['FUNCION', 'DEPARTAMENTO']] = \
    r1[['DEPARTAMENTO', 'FUNCION']].values

# INVERTIR RANGO > 148754 a final
rango_dest = df_nomina.loc[148754:, ['FUNCION', 'DEPARTAMENTO']]
rango_src  = df_nomina.loc[148754:, ['DEPARTAMENTO', 'FUNCION']]

n = min(len(rango_dest), len(rango_src))

df_nomina.loc[148754:148754+n-1, ['FUNCION', 'DEPARTAMENTO']] = \
    df_nomina.loc[148754:148754+n-1, ['DEPARTAMENTO', 'FUNCION']].values

# REVISAR FILAS CON DEPARTAMENTO INCORRECTO
cond = df_nomina['DEPARTAMENTO'].isin([
    'ENC AUDITORIA INTERNA',
    'AUDITOR II',
    'SUPERVISOR (A)',
    'COORDINADOR ACADEMICO',
])

# Corregir año erróneo desde la fila 134494
df_nomina.loc[134494:, 'ANIO'] = 2025

df_nomina.loc[cond, ['FUNCION', 'DEPARTAMENTO']] = \
    df_nomina.loc[cond, ['DEPARTAMENTO', 'FUNCION']].values

# Filtrado final
df_nomina.dropna(subset=['ANIO', 'MES_NUM', 'SUELDO_NOMINAL'], inplace=True)
df_nomina['ANIO'] = df_nomina['ANIO'].astype(int)
df_nomina['MES_NUM'] = df_nomina['MES_NUM'].astype(int)

print(f"ETL Completado.")

ETL Completado.


## 3. Construcción del Data Warehouse (Metodología Kimball)
Diseño del esquema en estrella centrado en el proceso de pago de nómina. Se generan:
* **Dimensiones con Llaves Subrogadas (`KEY_...`):** `DIM_TIEMPO` (Llave inteligente YYYYMM), `DIM_ORGANIZACION`, `DIM_CARGO`, `DIM_EMPLEADO` y `DIM_ESTATUS`.
* **Tabla de Hechos (`FACT_NOMINA`):** Centraliza las métricas de negocio (`monto_sueldo`, `sueldo_real`, `impuesto_renta`) y métricas de contexto económico (`valor_ipc`, `inflacion`), conectadas a las dimensiones mediante sus llaves primarias (`KEY_...`) al inicio de la tabla.
---

### GUÍA DE MÉTRICAS PARA EL CONTRALOR GENERAL:

1. **monto_sueldo_nominal_bruto**: 
   - Gasto público directo en nómina antes de deducciones.
   - Uso: Auditoría presupuestaria y control del gasto corriente.

2. **monto_sueldo_poder_adquisitivo_real**:
   - Valor del sueldo ajustado a la inflación histórica.
   - Uso: Evaluar si el ajuste salarial compensa el costo de vida real del servidor público.

3. **monto_retencion_isr_estimado**:
   - Cálculo aproximado del Impuesto Sobre la Renta retenido.
   - Uso: Proyección de retorno fiscal a la DGII proveniente de la nómina pública.

4. **indice_precios_consumidor_base_2020**:
   - Nivel general de precios reportado por el Banco Central.
   - Uso: Referencia macroeconómica para ajustes salariales.

5. **tasa_variacion_mensual_precios (VAR_MES)**:
   - Cambio porcentual de precios respecto al mes anterior.
   - Uso: Monitoreo de choques inflacionarios a corto plazo.

6. **tasa_inflacion_acumulada_anio (VAR_DIC)**:
   - Inflación acumulada desde Enero hasta el mes corriente.
   - Uso: Evaluar la pérdida de valor del presupuesto anual aprobado.

7. **tasa_inflacion_interanual_12m (VAR_12)**:
   - Inflación comparada con el mismo mes del año anterior.
   - Uso: Estándar principal para negociaciones de reajuste salarial anual.

8. **promedio_movil_inflacion_12m (PROM_12)**:
   - Promedio de las variaciones de los últimos 12 meses.
   - Uso: Suaviza la volatilidad para identificar tendencias de inflación estructural.

In [4]:
print("Construyendo Modelo Dimensional con nuevas reglas de negocio...")

df_stage = pd.merge(df_nomina, df_ipc_clean, on=['ANIO', 'MES_NUM'], how='left')
meses_inv_map = {v: k for k, v in meses_map.items()}

# DIM_TIEMPO
periodos = df_stage[['ANIO', 'MES_NUM']].drop_duplicates().sort_values(['ANIO', 'MES_NUM']).copy()

periodos['fecha_temp'] = pd.to_datetime(periodos['ANIO'].astype(str) + '-' + periodos['MES_NUM'].astype(str) + '-01')
periodos['fecha_fin_mes'] = periodos['fecha_temp'] + MonthEnd(0)

def obtener_estacion(mes):
    if mes in [12, 1, 2]: return 'INVIERNO'
    elif mes in [3, 4, 5]: return 'PRIMAVERA'
    elif mes in [6, 7, 8]: return 'VERANO'
    else: return 'OTOÑO'

dim_tiempo = pd.DataFrame()
dim_tiempo['KEY_TIEMPO'] = periodos['ANIO'] * 100 + periodos['MES_NUM']
dim_tiempo['fecha_iso'] = periodos['fecha_fin_mes'].dt.date
dim_tiempo['anio'] = periodos['ANIO']
dim_tiempo['mes_num'] = periodos['MES_NUM']
dim_tiempo['mes_nombre'] = periodos['MES_NUM'].map(meses_inv_map)
dim_tiempo['trimestre'] = periodos['fecha_fin_mes'].dt.quarter
dim_tiempo['estacion'] = periodos['MES_NUM'].apply(obtener_estacion) 
dim_tiempo['decada'] = (periodos['ANIO'] // 10) * 10
dim_tiempo['quinquenio'] = (periodos['ANIO'] // 5) * 5
dim_tiempo = dim_tiempo[['KEY_TIEMPO', 'fecha_iso', 'anio', 'mes_num', 'mes_nombre', 'trimestre', 'estacion', 'decada', 'quinquenio']]

# DIM_ORGANIZACION
dim_org = df_stage[['DEPARTAMENTO']].drop_duplicates().reset_index(drop=True)
dim_org['KEY_ORGANIZACION'] = dim_org.index + 1
dim_org = dim_org.rename(columns={'DEPARTAMENTO': 'departamento'})
dim_org = dim_org[['KEY_ORGANIZACION', 'departamento']]

# DIM_CARGO
dim_cargo = df_stage[['FUNCION']].drop_duplicates().reset_index(drop=True)
dim_cargo['KEY_CARGO'] = dim_cargo.index + 1
def categorizar_grupo(cargo):
    c = str(cargo).upper()
    if any(x in c for x in ['DIRECTOR', 'GERENTE', 'ENCARGADO', 'MINISTRO']): return 'NIVEL ESTRATEGICO'
    if any(x in c for x in ['COORDINADOR', 'SUPERVISOR']): return 'NIVEL TACTICO'
    if any(x in c for x in ['ANALISTA', 'AUDITOR', 'ABOGADO']): return 'NIVEL PROFESIONAL'
    return 'NIVEL OPERATIVO'
dim_cargo['grupo_ocupacional'] = dim_cargo['FUNCION'].apply(categorizar_grupo)
dim_cargo = dim_cargo.rename(columns={'FUNCION': 'cargo_nombre'})
dim_cargo = dim_cargo[['KEY_CARGO', 'cargo_nombre', 'grupo_ocupacional']]

# DIM_EMPLEADO
dim_emp = df_stage[['NOMBRE']].drop_duplicates().reset_index(drop=True)
dim_emp['KEY_EMPLEADO'] = dim_emp.index + 1
dim_emp = dim_emp.rename(columns={'NOMBRE': 'nombre_completo'})
dim_emp = dim_emp[['KEY_EMPLEADO', 'nombre_completo']]

# DIM_ESTATUS
dim_estatus = df_stage[['ESTATUS']].drop_duplicates().reset_index(drop=True)
dim_estatus['KEY_ESTATUS'] = dim_estatus.index + 1
dim_estatus = dim_estatus.rename(columns={'ESTATUS': 'descripcion_estatus'})
dim_estatus = dim_estatus[['KEY_ESTATUS', 'descripcion_estatus']]

# --- FACT_NOMINA ---
fact = df_stage.copy()
fact = pd.merge(fact, dim_org, left_on='DEPARTAMENTO', right_on='departamento', how='left')
fact = pd.merge(fact, dim_cargo, left_on='FUNCION', right_on='cargo_nombre', how='left')
fact = pd.merge(fact, dim_emp, left_on='NOMBRE', right_on='nombre_completo', how='left')
fact = pd.merge(fact, dim_estatus, left_on='ESTATUS', right_on='descripcion_estatus', how='left')

fact['KEY_TIEMPO'] = fact['ANIO'] * 100 + fact['MES_NUM']

# Cálculos previos
ipc_base = fact['IPC'].min()
fact['sueldo_real_calc'] = (fact['SUELDO_NOMINAL'] / fact['IPC']) * ipc_base
fact['impuesto_renta_calc'] = fact['SUELDO_NOMINAL'].apply(calcular_isr)

fact_nomina = fact[[
    'KEY_TIEMPO', 'KEY_ORGANIZACION', 'KEY_CARGO', 'KEY_EMPLEADO', 'KEY_ESTATUS', # FKs
    'SUELDO_NOMINAL', 
    'sueldo_real_calc', 
    'impuesto_renta_calc',
    'IPC', 
    'inflacion_mensual', 
    'inflacion_acumulada',
    'inflacion_interanual',
    'PROM_12'
]].copy()

nombres_metricas = {
    'SUELDO_NOMINAL': 'monto_sueldo_nominal_bruto',
    'sueldo_real_calc': 'monto_sueldo_poder_adquisitivo_real',
    'impuesto_renta_calc': 'monto_retencion_isr_estimado',
    'IPC': 'indice_precios_consumidor_base_2020',
    'inflacion_mensual': 'tasa_variacion_mensual_precios',
    'inflacion_acumulada': 'tasa_inflacion_acumulada_anio',
    'inflacion_interanual': 'tasa_inflacion_interanual_12m',
    'PROM_12': 'promedio_movil_inflacion_12m'
}

fact_nomina.rename(columns=nombres_metricas, inplace=True)
fact_nomina.index.name = 'KEY_NOMINA'
fact_nomina = fact_nomina.reset_index()

print(f"Fact Table Construida: {len(fact_nomina)} registros.")

Construyendo Modelo Dimensional con nuevas reglas de negocio...
Fact Table Construida: 152728 registros.


## 4. Carga y Persistencia de Datos
Almacenamiento del modelo dimensional final en una base de datos SQLite (`DW_Nomina.db`) y exportación de tablas individuales (`FACT` y `DIMs`) a formato CSV para su consumo en herramientas de visualización.

In [5]:
db_name = 'DW_Nomina.db'
conn = sqlite3.connect(db_name)

dim_tiempo.to_sql('DIM_TIEMPO', conn, if_exists='replace', index=False)
dim_org.to_sql('DIM_ORGANIZACION', conn, if_exists='replace', index=False)
dim_cargo.to_sql('DIM_CARGO', conn, if_exists='replace', index=False)
dim_emp.to_sql('DIM_EMPLEADO', conn, if_exists='replace', index=False)
dim_estatus.to_sql('DIM_ESTATUS', conn, if_exists='replace', index=False)
fact_nomina.to_sql('FACT_NOMINA', conn, if_exists='replace', index=False)

conn.close()

dim_tiempo.to_csv('DIM_TIEMPO.csv', index=False)
dim_org.to_csv('DIM_ORGANIZACION.csv', index=False)
dim_cargo.to_csv('DIM_CARGO.csv', index=False)
dim_emp.to_csv('DIM_EMPLEADO.csv', index=False)
dim_estatus.to_csv('DIM_ESTATUS.csv', index=False)
fact_nomina.to_csv('FACT_NOMINA.csv', index=False)

print("--- ETL FINALIZADO ---")

--- ETL FINALIZADO ---
